In [3]:
import cv2
import pytesseract
import time
import numpy as np
from PIL import Image,ImageDraw
import pyautogui
import py.variable as v
import py.map as m
import py.dofus_action as da
import socket
import sys
from pathlib import Path
import models.predict as predict
from scipy import ndimage 
import win32api

In [ ]:
time.sleep(3)
while True :
    x,y = pyautogui.position()
    if win32api.GetKeyState(0x01)<0 :
        print("x : ", x,"y :", y)
        time.sleep(0.3)


In [15]:
dep_droite_2pm = (85,0)
dep_gauche_2pm = (-85,0)
dep_haut_2pm = (0,-45)
dep_bas_2pm = (0,45)

#/>
dep_haut_1pm = (45,-25)
dep_droite_1pm = (40,20)
dep_bas_1pm = (-45,25)
dep_gauche_1pm = (-40,-20)
model_fight = "models/model_fight_i3/model.pb"
label_fight = "models/model_fight_i3/labels.txt"
od_model_fight = predict.load_model(MODEL_FILENAME = model_fight,LABELS_FILENAME = label_fight)


In [36]:
time.sleep(3)
image = pyautogui.screenshot(region=v.region_gameplay)
w,h = image.size
predictions = od_model_fight.predict_image(image)

list_ennemis = [da.list_click(x["boundingBox"],w, h,1) for x in predictions if  ((x["tagName"] == "ennemi") and (x["probability"]) > 0.5)] 
list_allie = [da.list_click(x["boundingBox"],w, h,1) for x in predictions if  ((x["tagName"] == "allie") and (x["probability"]) > 0.1)] 
print(list_ennemis, list_allie)
mon_perso = list_allie[0]
plus_proche_ennemi = da.minDistance(mon_perso, list_ennemis)
da.dofus_click(plus_proche_ennemi[0],plus_proche_ennemi[1], 1,0)
print(mon_perso,plus_proche_ennemi)
deplacer_mon_perso(mon_perso,plus_proche_ennemi)
da.Distance(mon_perso,plus_proche_ennemi)

[(1111.5, 328.6)] [(975.4, 395.7)]
(975.4, 395.7) (1111.5, 328.6)
NE


151.74195201064208

In [ ]:
dep_O_2pm = (85,0)
dep_E_2pm = (-85,0)
dep_N_2pm = (0,-45)
dep_S_2pm = (0,45)

dep_NE_2pm = (90,-50)
dep_SE_2pm = (80,40)
dep_SO_2pm = (-90,50)
dep_NO_2pm = (-80,-40)
#/>
dep_NE_1pm = (45,-25)
dep_SE_1pm = (40,20)
dep_SO_1pm = (-45,25)
dep_NO_1pm = (-40,-20)

def deplacer_mon_perso(perso, ennemi, pm) :
    if da.Distance(mon_perso,plus_proche_ennemi) < 160 :
        print("ennemi a moins de 4 po")
        return None
    if pm == 0 :
        return perso
    while pm >= 2 : 
        direction = direction_ennemi(perso, ennemi)
        match direction :
            case "N" : 
                perso = add_deplacement(perso, dep_N_2pm)
                pm -= 2
            case "O" :
                perso = add_deplacement(perso, dep_O_2pm)
                pm -= 2
            case "S" :
                perso = add_deplacement(perso, dep_S_2pm)
            case "E" :
                perso = add_deplacement(perso, dep_E_2pm)
                pm -= 2
            case "NE" :
                perso = add_deplacement(perso, dep_NE_2pm)
                pm -= 2
            case "NO" :
                perso = add_deplacement(perso, dep_NO_2pm)
                pm -= 2
            case "SO" :
                perso = add_deplacement(perso, dep_SO_2pm)
                pm -= 2
            case "SE" :
                perso = add_deplacement(perso, dep_SE_2pm)
                pm -= 2
    return perso

In [37]:
def add_deplacement(perso, deplacement) :
    perso_x , perso_y = perso
    deplacement_x , deplacement_y = deplacement
    return ((float(perso) + float(deplacement_x)), (float(perso_y) + float(deplacement_y)))
            
def direction_ennemi(mon_perso,ennemi) : 
    perso_x , perso_y  = [float(pos) for pos in mon_perso]
    ennemi_x ,ennemi_y = [ float(dep) for dep in ennemi]
    
    if ((perso_y - 7) <= ennemi_y ) and (ennemi_y <= (perso_y +7)) :
        if  ennemi_x < perso_x :
            print("O")
            return "O"
        else :
            print("E")
            return "E"
    elif ((perso_x - 7 ) <= ennemi_x) and ( ennemi_x <= (perso_x+7)) :
        if ennemi_y < perso_y:
            print("N")
            return "N"
        else :
            print("S")
            return "S"
    elif (ennemi_y <= (perso_y - 7)) and (ennemi_x <= (perso_x -7)) :
        print("NO")
        return "NO"
    elif (ennemi_y >= (perso_y + 7)) and (ennemi_x <= (perso_x -7)) :
        print("SO")
        return "SO"
    elif (ennemi_y <= (perso_y - 7)) and (ennemi_x >= (perso_x +7)) :
        print("NE")
        return "NE"
    elif (ennemi_y >= (perso_y + 7)) and (ennemi_x >= (perso_x +7)) :
        print("SE")
        return "SE"